In [28]:
#Importing Packages and Checking GPU
import tensorflow as tf
from tensorflow.keras import layers,models
import numpy as np
from scipy import io
from tensorflow.keras.callbacks import EarlyStopping

GPUDevices = tf.config.list_physical_devices('GPU')

if GPUDevices:
	print("TensorFlow is using the GPU")
	for i,gpu in enumerate(GPUDevices):
		print(f"GPU {i} Found: {gpu.name}")
else:
	print("No GPU Devices found. TensorFlow is using the CPU")

No GPU Devices found. TensorFlow is using the CPU


In [29]:
#Loading DataSet
print("Loading EMNIST DataSet")
dataset = io.loadmat('datasets/emnist-letters.mat')
struct_dataset = dataset['dataset'][0,0]

TrainImages, TrainLabels = struct_dataset['train'][0,0]['images'], struct_dataset['train'][0,0]['labels']
TestImages, TestLabels = struct_dataset['test'][0,0]['images'], struct_dataset['test'][0,0]['labels']

TrainImages = TrainImages / 255.0
TestImages = TestImages / 255.0

TrainLabels = TrainLabels - 1
TestLabels = TestLabels - 1

TrainImages = TrainImages.reshape(len(TrainImages), 28, 28, 1)
TestImages = TestImages.reshape(len(TestImages), 28, 28, 1)

print("DataSet Loaded")
print(f"Train Images Shape: {TrainImages.shape}")
print(f"Train Labels Shape: {TrainLabels.shape}")
print(f"Test Images Shape: {TestImages.shape}")
print(f"Test Labels Shape: {TestLabels.shape}")

Loading EMNIST DataSet
DataSet Loaded
Train Images Shape: (124800, 28, 28, 1)
Train Labels Shape: (124800, 1)
Test Images Shape: (20800, 28, 28, 1)
Test Labels Shape: (20800, 1)


In [30]:
#Creating Model
print("Creating Model")

model = models.Sequential()

#Layer 1
model.add(layers.Conv2D(32 , (3,3),input_shape = (28,28,1),padding='same',use_bias="false"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

#Layer 2
model.add(layers.Conv2D(32 , (3,3),padding='same',use_bias="false"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))

#Layer 3
model.add(layers.Conv2D(64 , (3,3),padding='same',use_bias="false"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))

#Layer 4
model.add(layers.Conv2D(64 , (3,3),padding='same',use_bias="false"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))

#Layer 5
model.add(layers.Conv2D(64 , (3,3),padding='same',use_bias="false"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
model.add(layers.MaxPooling2D(pool_size = (2,2)))

#Flatten
model.add(layers.Flatten())

#Layer 6
model.add(layers.Dense(64))
model.add(layers.Activation("relu"))

#Layer 7
model.add(layers.Dense(64))
model.add(layers.Activation("relu"))

#Dropout
model.add(layers.Dropout(0.2))

#Layer 8
model.add(layers.Dense(26))
model.add(layers.Activation("softmax"))

model.compile(loss = "sparse_categorical_crossentropy" , optimizer = "adam" , metrics = ["accuracy"])

print("Model Created")
print("Model Summary: ")
model.summary()

Creating Model
Model Created
Model Summary: 


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_25 (Conv2D)              │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_25          │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_38 (Activation)      │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_26 (Conv2D)              │ (None, 14, 14, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_26          │ (None, 14, 14, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_39 (Activation)      │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_27 (Conv2D)              │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_40 (Activation)      │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_28 (Conv2D)              │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_41 (Activation)      │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_29 (Conv2D)              │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_42 (Activation)      │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 64)             │       200,768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_43 (Activation)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_44 (Activation)      │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 309,562 (1.18 MB)

 Trainable params: 309,050 (1.18 MB)

 Non-trainable params: 512 (2.00 KB)

In [31]:
#Training and Saving model

print("Training Model")
EarlyStop = EarlyStopping(
	monitor = 'val_accuracy',
	patience = 5,
	verbose = 1,
	restore_best_weights = 1
)

Training = model.fit(TrainImages,TrainLabels,
					 batch_size = 128,
					 epochs = 100,
					 validation_data = (TestImages,TestLabels),
					 callbacks = [EarlyStop])
print("Training Complete")

model.save("EMNISTModel.keras")
print("Model Saved")

Training Model
Epoch 1/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 37s 37ms/step - accuracy: 0.8066 - loss: 0.6340 - val_accuracy: 0.9077 - val_loss: 0.2870
Epoch 2/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 38s 39ms/step - accuracy: 0.9140 - loss: 0.2687 - val_accuracy: 0.9279 - val_loss: 0.2218
Epoch 3/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 40s 41ms/step - accuracy: 0.9272 - loss: 0.2255 - val_accuracy: 0.9118 - val_loss: 0.2740
Epoch 4/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 39s 40ms/step - accuracy: 0.9331 - loss: 0.2041 - val_accuracy: 0.9261 - val_loss: 0.2384
Epoch 5/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 41s 42ms/step - accuracy: 0.9369 - loss: 0.1911 - val_accuracy: 0.9401 - val_loss: 0.1840
Epoch 6/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 39s 40ms/step - accuracy: 0.9408 - loss: 0.1767 - val_accuracy: 0.9376 - val_loss: 0.1916
Epoch 7/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 39s 40ms/step - accuracy: 0.9435 - loss: 0.1675 - val_accuracy: 0.9351 - val_loss: 0.2049
Epoch 8/100
975/975 ━━━━━━━━━━━━━━━━━━━━ 40s 41ms/step - accuracy: 0

In [32]:
#Testing Model

print("Testing Model with inbuilt dataset")
TestLoss,TestAcc = model.evaluate(TestImages,TestLabels)
print(TestImages.shape)
print(TestImages.dtype)
print(np.min(TestImages), np.max(TestImages))
print(f"Mean: {np.mean(TestImages)}")
print(f"Test Accuracy: {TestAcc}")
print(f"Test Loss: {TestLoss}")

Testing Model with inbuilt dataset
650/650 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9434 - loss: 0.1747
(20800, 28, 28, 1)
float64
0.0 1.0
Mean: 0.1721735874638317
Test Accuracy: 0.9434134364128113
Test Loss: 0.17468087375164032
